In [204]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from analysts_tools.redash_methods import *
    #Frubana libraries
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging

In [205]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""

city_code = 'VCP' # IMPORTAN TO CHANGE THE CITY

# Set today's date to the first day of the next month
today = datetime.today()
first_day_next_month = (today.replace(day=1) + timedelta(days=32)).replace(day=1)

# Calculate non_buyers_date (todays date - 1 month)
non_buyers_date = (first_day_next_month - timedelta(days=60)).replace(day=1)
customer_date_desde = (first_day_next_month - timedelta(days=60)).replace(day=1)


# Convert the dates to strings in the desired format
todays_date = first_day_next_month.strftime('%Y-%m-%d')
non_buyers_date = non_buyers_date.strftime('%Y-%m-%d')
customer_date_desde = customer_date_desde.strftime('%Y-%m-%d')

print(todays_date) # Output: e.g. 2023-04-01
print(non_buyers_date) # Output: e.g. 2023-03-01
print(customer_date_desde) # Output: e.g. 2023-03-01

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

2024-08-01
2024-06-01
2024-06-01


In [206]:
query="""
With microsegment as (

SELECT DISTINCT c.customer_id AS user_id,
                c.site_type AS region_code_ms,
                
                (case 
                    -- Colombia
                    when m.microsegment_id not in (1,3,4,28,5,6,7,8,2) and c.site_type = 'BOG' then 'Sin Microsegmento'
                    when m.microsegment_id not in (1,3,4,28,5,11,10,9,2,12) and c.site_type = 'BAQ' then 'Sin Microsegmento'
                    when m.microsegment_id not in (3,4,28,1,24,11,5,2,27) and c.site_type = 'MDE' then 'Sin Microsegmento'
    
                    -- Mexico
                    when m.microsegment_id not in (1,13,14,30,15,16,29,17,6,7,2) and c.site_type = 'CMX' then 'Sin Microsegmento'
                    when m.microsegment_id not in (1, 13,14,30, 15,16,29, 6, 33, 2, 31) and c.site_type = 'GDL' then 'Sin Microsegmento'
                    when m.microsegment_id not in (13,14,30, 15,16,29, 32,2,6,33) and c.site_type = 'PBC' then 'Sin Microsegmento'
                    
                    -- Brasil
                    when m.microsegment_id not in (21,34,38,37,35,36,39,26) and c.site_type in ('SPO','BHZ','CWB','VCP') then 'Sin Microsegmento'
                        
                else m.description end) as microsegment_ok,
                
                m.microsegment_id,
                mo.name as ms_method
                
                
FROM postgres_main_federate."growth_microsegmentation.customer_microsegment" AS cm
INNER JOIN postgres_main_federate."growth_microsegmentation.microsegment" m ON cm.microsegment_id = m.id
INNER JOIN postgres_main_federate."growth_microsegmentation.customer" c ON cm.customer_id = c.id
INNER JOIN postgres_main_federate."growth_microsegmentation.models" mo ON mo.id = cm.model_type_id


)

select m.user_id,
m.region_code_ms,
--m.microsegment_id,

(case 
        when m.microsegment_ok in ('Corrientazo c/desayuno','Corrientazo s/desayuno','Tienda/Restaurante') then 'Corrientazo' 
        when m.microsegment_ok in ('Fonda c/desayuno','Fonda s/desayuno') then 'Fonda'
        when m.microsegment_ok in ('Taquería en calle','Taquería en local') then 'Taquería'

else m.microsegment_ok end) as microsegment,

m.ms_method

from microsegment m
where m.microsegment_ok != 'Sin Microsegmento'
"""

segmentos_fulll = read_connection_data_warehouse.runQuery(query)

segmentos_fulll.columns = ['customer_id', 'city', 'microsegment','NA']
segmentos_fulll = segmentos_fulll[['customer_id', 'city', 'microsegment']]

### Loading COSTUMERS INFO

In [207]:
users_eliminar_kam = get_fresh_query_result("https://internal-redash.federate.frubana.com/",150812,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
users_eliminar = users_eliminar_kam.loc[users_eliminar_kam.registered_city == city_code]['customer_id'].unique().tolist()

users_eliminar_pro = get_fresh_query_result("https://internal-redash.federate.frubana.com/",206229,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
users_eliminar_pro[['user_id']] = users_eliminar_pro[['user_id']].astype(int)
users_eliminar2 = users_eliminar_pro.loc[users_eliminar_pro.region_code == city_code]['user_id'].unique().tolist()

200
200


In [208]:
users_eliminar = list(set(users_eliminar + users_eliminar2))

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [209]:
#//Remove skus 
#It is used because some SKUs can get in trouble with supplier for the SD or to exclude them for another activity   
skus_remove = GoogleSheet("1Z0T_c36oRHmOHIgEG8_ANcwFykaHgXGAizYAYCKCqtY")
skus_remove = skus_remove.get_as_dataframe('SKUS')

skus_remove = skus_remove['sku_id'].unique().astype(int).tolist()

In [210]:
query = """
SELECT DISTINCT
    split_part(bs.upc, '-', 1) as region,
    bs.sku_id

FROM postgres_broadleaf_federate."broadleaf.blc_sku"            bs 
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product"   bp ON bp.product_id = bs.addl_product_id

WHERE bs.archived = 'N'
    AND bs.sndbx_id is NULL
    AND bp.archived = 'N'
    AND bs.catalog_disc < 0
    AND ((bp.archived = 'N' AND bp.sndbx_id is NULL) OR (bp.archived is NULL))
"""
frida_products = read_connection_data_warehouse.runQuery(query)

In [211]:
good_skus = frida_products.sku_id.unique()

In [212]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [213]:
data_ventas = data_ventas[(~data_ventas['sku_id'].isin(skus_remove)) & (data_ventas['sku_id'].isin(good_skus))]

In [214]:
segmentos_full = segmentos_fulll[segmentos_fulll.city == city_code].copy()

In [215]:
#//customers for offers */(has further uses)*/
microsegmentacion_df = segmentos_full[(~segmentos_full['customer_id'].isin(users_eliminar))]
data_ventas = data_ventas[(~data_ventas['customer_id'].isin(users_eliminar))]

### Create the MS "Other" for BAU 

In [216]:
data_full = data_ventas.merge(microsegmentacion_df[['customer_id','microsegment']], how = 'left', on = 'customer_id')

In [217]:
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [218]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Boteco,97
1,Grill/Churrascaria/Espetinho,68
2,Japonesa/Internacional,38
3,Lanches,334
4,Other,188
5,Padaria/Cafe/Bolos,296
6,Pizzaria/Pastelaria/Italiano,96
7,Pratos/Refeições,382
8,Sucos/Açaí,90


In [219]:
segments_validation = data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

In [220]:
for ms in segments_validation[segments_validation.customer_id < 180]['microsegment'].unique():
    data_full['microsegment'] = data_full['microsegment'].str.replace(ms,'Other')

In [221]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Lanches,334
1,Other,577
2,Padaria/Cafe/Bolos,296
3,Pratos/Refeições,382


In [222]:
#Get the same variable to do the process job
data_ventas = data_full

### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [223]:
def ms_data(city_code):
  
    paramss_csv=GoogleSheet("1k0ckgkLddxc5lY8NUVGCkvDLTuulLaJm-N535oMNZNw")
    city_paramss = paramss_csv.get_as_dataframe(city_code)
    
    parametros = {}
    parametros[city_code] = {}
    
    for i,ms in enumerate(city_paramss.iloc[:,4:].columns):
        parametros[city_code][ms] = [(value[1],value[2],value[3],value[4+i]) for value in city_paramss.values if value[4+i]>0]
        
    return parametros

In [224]:
parametros = ms_data(city_code)

In [225]:
def normalize_text(raw_text):
    """
    Removes common accent characters and normalize text.
    """
    raw_text = raw_text.replace(" ", "")
    raw_text = raw_text.replace("�", "")
    raw_text = raw_text.replace("/", "")
    raw_text = re.sub(r'\s*', '', raw_text)
    raw_text = re.sub(r"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(r"[èéêë]", 'e', raw_text)
    raw_text = re.sub(r"[ìíîï]", 'i', raw_text) 
    raw_text = re.sub(r"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(r"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(r"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(r"[ß]", 'ss', raw_text)
    raw_text = re.sub(r"[ñ]", 'n', raw_text)
    raw_text = re.sub(r"[ç]", 'c', raw_text)
    raw_text = raw_text.lower()
    return raw_text

In [226]:
#Sacar de aca los microsegmentos
msg = data_full.microsegment.unique()
msg

array(['Pratos/Refeições', 'Lanches', 'Other', 'Padaria/Cafe/Bolos'],
      dtype=object)

### END

# Aca Empieza la Experimentacion

In [227]:
from sklearn.model_selection import train_test_split 

In [228]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_segment_list = [] #//for procesing

In [229]:
data_ventas['subcat'] = data_ventas['subcat'].str.capitalize()

In [230]:
#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
aux_df = pd.DataFrame()
aux_df['aux']=6000*[0]

for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    #//Customers per microsegment
    customers_ms = data_ventas[(data_ventas['microsegment'] == microsegment)]['customer_id'].unique()
    #//Customers to give them the skus
    customers_for_skus_ms =  data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == microsegment) & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()
    
    #----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_PILOTO" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    rand=1
    customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state=rand)
    
    while abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())) > 0.05:
        #print(data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean(),data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean(),abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())))
        rand+=1
        customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state = rand)
        
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_1)].copy() #le quite la s!!
    
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()
    
    subcats = list(map(lambda x: x[0], parametros[city_code][microsegment]))
    max_uses = list(map(lambda x: int(x[1]), parametros[city_code][microsegment]))
    max_order = list(map(lambda x: int(x[2]), parametros[city_code][microsegment]))
    discount = list(map(lambda x: int(x[3]), parametros[city_code][microsegment]))

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
            non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                        limite=0.75, data_ventas_skus = data_skus)

    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)

        lst.append(d)
        
    #//appending to global list    
        global_segment_list.append(d)
    
    print(f'Done {city_code} {ms_name}')

#----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------


#----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_CONTROL" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_2)].copy() #le quite la s!!
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    print('//-------------------------------------------------------------------------------')
    print(f'{microsegment} Experimento50%')
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    e, f, g = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus)
      
    print(f.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(e)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in f.columns:
        id_list = f[seg].dropna().astype(int)
        h = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)
        
        lst.append(h)
        
        
    #//appending to global list    
        global_segment_list.append(h)
    
    print(f'Done {city_code} {ms_name}')
    #----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#//End of big loop and start of data bundle 

//-------------------------------------------------------------------------------
Pratos/Refeições
306
Uso de data externa
20240801_VCP_pratosrefeicoes_piloto_NB_cafechocolateseinfusoes_W31         215
20240801_VCP_pratosrefeicoes_piloto_NB_tuberculos_W31                      149
20240801_VCP_pratosrefeicoes_piloto_NB_bomboniereesnacks_W31               290
20240801_VCP_pratosrefeicoes_piloto_NB_frutas_W31                          176
20240801_VCP_pratosrefeicoes_piloto_NB_suinos_W31                          264
20240801_VCP_pratosrefeicoes_piloto_NB_temperos_W31                        138
20240801_VCP_pratosrefeicoes_piloto_NB_detergentesabaoelava-louas_W31      145
20240801_VCP_pratosrefeicoes_piloto_NB_embutidos_W31                       230
20240801_VCP_pratosrefeicoes_piloto_NB_massasemolhos_W31                   142
20240801_VCP_pratosrefeicoes_piloto_NB_aucaradoantesedoces_W31             165
20240801_VCP_pratosrefeicoes_piloto_NB_docesesobremesas_W31                304
20240801

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


Uso de data externa
20240801_VCP_lanches_control_NB_cafechocolateseinfusoes_W31         57
20240801_VCP_lanches_control_NB_tuberculos_W31                      48
20240801_VCP_lanches_control_NB_bomboniereesnacks_W31               65
20240801_VCP_lanches_control_NB_frutas_W31                          47
20240801_VCP_lanches_control_NB_suinos_W31                          59
20240801_VCP_lanches_control_NB_temperos_W31                        58
20240801_VCP_lanches_control_NB_detergentesabaoelava-louas_W31      49
20240801_VCP_lanches_control_NB_embutidos_W31                       55
20240801_VCP_lanches_control_NB_massasemolhos_W31                   57
20240801_VCP_lanches_control_NB_docesesobremesas_W31                65
20240801_VCP_lanches_control_NB_frutaselegumescongelados_W31        64
20240801_VCP_lanches_control_NB_graos_W31                           64
20240801_VCP_lanches_control_NB_refrigeranteseenergeticos_W31       32
20240801_VCP_lanches_control_NB_papeis_W31               

## HERE WE CREATE THE FILES FOR R&D - DESCOMENTAR PARA ENVIAR A R&D Y !!!! COMENTAR PARA CARGAR FINALES !!!!

In [231]:
# customers =  pd.DataFrame()
# for column,i in enumerate(aux_df.columns):
#     #if ('verduras' in i or 'frutas' in i or 'arroz' in i or 'aceitesygrasas' in i or 'harinasymezclas' in i or 'azucaryendulzantes' in i or 'pollo' in i or 'huevos' in i) and not 'frutasjugo' in i:
#     #if ('verduras' in i or 'frutas' in i) and not 'frutasjugo' in i:
#     customers[i] = aux_df[i]

In [232]:
# customers.iloc[:7000].to_csv(f'Growth/SD/export_data/MS/csv_{city_code}_CLIENTS_PRUEBA_R&D.csv', index = False)

# with open(f'Growth/SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#        json.dump(global_segment_list, outfile)

## HERE WE LOAD THE FILES OF R&D TO MATCH THE CUSTOMERS OF THEIR INTELLIGENCE 

IN THE FIRST PART, WE SHOULD CHANGE THE MONTH i.e

aux_seg = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_DICIEMBRE_R&D.csv") ---> aux_seg = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D.csv")

THEN CHANGE THE DATE AND THE W, WE WILL SEE THAT IN THE PREVIOUS CHARGE ABOVE i.e.

20231201_BOG_empanadasarepas_control_NB_leches_W48   

Then we now:


aux_seg['offer_name'] = f"20231101_{city_code}_" + aux_seg.ms + "_" + aux_seg.final_group2 + "_NB_" + aux_seg.subcategory + "_W43"

aux_seg["quartile_name"] = f"20231101_{city_code}_" + aux_seg.subcategory + "_" + aux_seg.quartile + "_" + aux_seg.case

become:

aux_seg['offer_name'] = f"20231201_{city_code}_" + aux_seg.ms + "_" + aux_seg.final_group2 + "_NB_" + aux_seg.subcategory + "_W48"

aux_seg["quartile_name"] = f"20231201_{city_code}_" + aux_seg.subcategory + "_" + aux_seg.quartile + "_" + aux_seg.case



In [233]:
aux_seg = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_AGOSTO_R&D.csv")
aux_seg["customer_source_id"] = aux_seg.customer_source_id.astype(int)

aux_seg["final_group2"] = ["piloto" if x == "Treatment" else "control" for x in aux_seg.final_group]
aux_seg['offer_name'] = f"20240801_{city_code}_" + aux_seg.ms + "_" + aux_seg.final_group2 + "_NB_" + aux_seg.subcategory + "_W31"

aux_seg["quartile"] = ["Q0" if x == "-" else x for x in aux_seg.quartile.values]
aux_seg["quartile_name"] = f"20240801_{city_code}_" + aux_seg.subcategory + "_" + aux_seg.quartile + "_" + aux_seg.case

aux_seg2 = aux_seg.groupby(by = ["offer_name"])["customer_source_id"].apply(list).reset_index()
aux_seg3 = aux_seg.groupby(by = ["quartile_name"])["customer_source_id"].apply(list).reset_index()

In [234]:
aux_seg.head()

,subcategory,customer_source_id,ms,previous_group,case,quartile,random,final_group,final_group2,offer_name,quartile_name
0,aguaseisotonicos,153508896,lanches,Treatment,positivity,Q4,0,Treatment,piloto,20240801_VCP_lanches_piloto_NB_aguaseisotonico...,20240801_VCP_aguaseisotonicos_Q4_positivity
1,aguaseisotonicos,286422209,other,Treatment,positivity,Q4,0,Treatment,piloto,20240801_VCP_other_piloto_NB_aguaseisotonicos_W31,20240801_VCP_aguaseisotonicos_Q4_positivity
2,aguaseisotonicos,352509000,pratosrefeicoes,Treatment,positivity,Q4,0,Treatment,piloto,20240801_VCP_pratosrefeicoes_piloto_NB_aguasei...,20240801_VCP_aguaseisotonicos_Q4_positivity
3,aguaseisotonicos,362035232,pratosrefeicoes,Treatment,positivity,Q4,0,Treatment,piloto,20240801_VCP_pratosrefeicoes_piloto_NB_aguasei...,20240801_VCP_aguaseisotonicos_Q4_positivity
4,aguaseisotonicos,142986347,other,Treatment,positivity,Q4,0,Treatment,piloto,20240801_VCP_other_piloto_NB_aguaseisotonicos_W31,20240801_VCP_aguaseisotonicos_Q4_positivity


In [235]:
import calendar
date_end = datetime.strptime(todays_date, "%Y-%m-%d")
end_date = (date_end.replace(day = calendar.monthrange(date_end.year, date_end.month)[1]) + timedelta(days = 0)).strftime("%Y-%m-%d")

In [236]:
lst = []
for x in aux_seg2.values:
    h = {'name':x[0], 'customersIds': list(x[1]),"expiresAt": end_date}
    lst.append(h)      
          
global_segment_list = lst

In [237]:
lst2 = []
for x in aux_seg3.values:
    h = {'name':x[0], 'customersIds': list(x[1]),"expiresAt": end_date}
    lst2.append(h)

# Aca tengo que volver a cargar el archivo de arriba y dejarlo como lista para traer lo otro

In [238]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20240801_VCP_pratosrefeicoes_piloto_NB_cafecho...,pratosrefeicoes_piloto_NB_cafechocolateseinfusoes,8,true,01-08-2024,31-08-2024,0,1,None,576222#576226#621291#430791#625317
1,20240801_VCP_pratosrefeicoes_piloto_NB_tubercu...,pratosrefeicoes_piloto_NB_tuberculos,8,true,01-08-2024,31-08-2024,0,1,None,434949#434919#434937#434935
2,20240801_VCP_pratosrefeicoes_piloto_NB_bomboni...,pratosrefeicoes_piloto_NB_bomboniereesnacks,8,true,01-08-2024,31-08-2024,0,1,None,652957
3,20240801_VCP_pratosrefeicoes_piloto_NB_frutas_W31,pratosrefeicoes_piloto_NB_frutas,8,true,01-08-2024,31-08-2024,0,1,None,434987#434895#434925#434983#575766#435001#435005
4,20240801_VCP_pratosrefeicoes_piloto_NB_suinos_W31,pratosrefeicoes_piloto_NB_suinos,8,true,01-08-2024,31-08-2024,0,1,None,483989#483953#622149#621479


## Crear segments offers y segments types

In [239]:
import http.client
import json

def create_segments(segment_list):
    
    conn = http.client.HTTPSConnection(f"{city_code}.frubana.com")
    payload = json.dumps(segment_list)
    headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; LOGIN_SITE=spo; AWSALB=wL3xN1Ur+mhBc+wMCC78ce1tF/HmjkUUEMLIV5soLFplvt6xIihLPOltXuYXFDicZ8aVCGObuaC6RceIQjPjxd0Rda+D3F5/hobGvtf2oJb00dVJQIoNiX/Aqh48; AWSALBCORS=wL3xN1Ur+mhBc+wMCC78ce1tF/HmjkUUEMLIV5soLFplvt6xIihLPOltXuYXFDicZ8aVCGObuaC6RceIQjPjxd0Rda+D3F5/hobGvtf2oJb00dVJQIoNiX/Aqh48',
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001", payload, headers)
    res = conn.getresponse()
    data = res.read()
    print(data.decode("utf-8"))

    sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, data.decode("utf-8")[32:].strip('][').replace('"', '').split(',')))]
    time.sleep(15)
    return sgt

In [240]:
len_seg =int(len(global_segment_list)/10)
len_seg_list = [global_segment_list[:len_seg],global_segment_list[len_seg:len_seg*2],global_segment_list[len_seg*2:len_seg*3],global_segment_list[len_seg*3:len_seg*4],global_segment_list[len_seg*4:len_seg*5],global_segment_list[len_seg*5:len_seg*6],global_segment_list[len_seg*6:len_seg*7],global_segment_list[len_seg*7:len_seg*8],global_segment_list[len_seg*8:len_seg*9],global_segment_list[len_seg*9:]]

In [241]:
#offers
sgt=[]

for i in len_seg_list:
    sgt.append(create_segments(i))
    
sgt_end =[item for lista in sgt for item in lista]

Customer segments were created: [781197-20240801_VCP_lanches_control_NB_aguaseisotonicos_W31, 781196-20240801_VCP_lanches_control_NB_arroz_W31, 781199-20240801_VCP_lanches_control_NB_azeitesoleosevinagres_W31, 781198-20240801_VCP_lanches_control_NB_cafechocolateseinfusoes_W31, 781200-20240801_VCP_lanches_control_NB_carnevermelha_W31, 781501-20240801_VCP_lanches_control_NB_cervejas_W31, 781502-20240801_VCP_lanches_control_NB_condimentos_W31, 781503-20240801_VCP_lanches_control_NB_conservaseenlatados_W31, 781504-20240801_VCP_lanches_control_NB_desengordurantes_W31, 781505-20240801_VCP_lanches_control_NB_desinfetantesesanitizantes_W31, 781506-20240801_VCP_lanches_control_NB_destilados_W31, 781507-20240801_VCP_lanches_control_NB_detergentesabaoelava-louas_W31, 781508-20240801_VCP_lanches_control_NB_embutidos_W31, 781509-20240801_VCP_lanches_control_NB_farinhasemisturas_W31, 781510-20240801_VCP_lanches_control_NB_frango_W31, 781511-20240801_VCP_lanches_control_NB_frutas_W31, 781512-20240801

In [242]:
len_seg2 =int(len(lst2)/10)
len_seg_list2 = [lst2[:len_seg2],lst2[len_seg2:len_seg2*2],lst2[len_seg2*2:len_seg2*3],lst2[len_seg2*3:len_seg2*4],lst2[len_seg2*4:len_seg2*5],lst2[len_seg2*5:len_seg2*6],lst2[len_seg2*6:len_seg2*7],lst2[len_seg2*7:len_seg2*8],lst2[len_seg2*8:len_seg2*9],lst2[len_seg2*9:]]

In [243]:
#types
sgt2=[]

for i in len_seg_list2:
    sgt2.append(create_segments(i))
    
sgt_end2 =[item for lista in sgt2 for item in lista]

Customer segments were created: [781682-20240801_VCP_aguaseisotonicos_Q0_positivity, 781683-20240801_VCP_aguaseisotonicos_Q1_positivity, 781684-20240801_VCP_aguaseisotonicos_Q2_positivity, 781686-20240801_VCP_aguaseisotonicos_Q3_positivity, 781685-20240801_VCP_aguaseisotonicos_Q4_positivity, 781687-20240801_VCP_arroz_Q0_positivity, 781688-20240801_VCP_arroz_Q1_positivity, 781689-20240801_VCP_arroz_Q2_positivity, 781690-20240801_VCP_arroz_Q3_positivity, 781691-20240801_VCP_arroz_Q4_positivity, 781692-20240801_VCP_aucaradoantesedoces_Q0_positivity, 781693-20240801_VCP_aucaradoantesedoces_Q1_positivity, 781694-20240801_VCP_aucaradoantesedoces_Q2_positivity, 781695-20240801_VCP_aucaradoantesedoces_Q3_positivity, 781696-20240801_VCP_aucaradoantesedoces_Q4_positivity, 781697-20240801_VCP_azeitesoleosevinagres_Q0_positivity, 781698-20240801_VCP_azeitesoleosevinagres_Q1_positivity, 781699-20240801_VCP_azeitesoleosevinagres_Q2_positivity, 781700-20240801_VCP_azeitesoleosevinagres_Q3_positivity]

In [244]:
#unir offers con segments
for i in global_offer_csv.offer_name:
    for j,z in sgt_end:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j

In [245]:
global_offer_csv = global_offer_csv.loc[~global_offer_csv.customer_segment_id.isnull()].reset_index(drop=True)
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('discount','max_uses_per_order')]=1
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('start_date','end_date')]='01-01-2028'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('end_date')]='02-01-2028'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('sku_id')]= global_offer_csv.loc[~global_offer_csv['offer_name'].str.contains("piloto", case=False), ('sku_id')].tolist()

In [246]:
global_offer_csv['segment_behaviour'] = 'INCLUDED'
global_offer_csv['offer_type'] = 'SUPER_DISCOUNT'
global_offer_csv['supplier_discount'] = 0

In [247]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id,segment_behaviour,offer_type,supplier_discount
0,20240801_VCP_pratosrefeicoes_piloto_NB_cafecho...,pratosrefeicoes_piloto_NB_cafechocolateseinfusoes,8,true,01-08-2024,31-08-2024,0,1,781548,576222#576226#621291#430791#625317,INCLUDED,SUPER_DISCOUNT,0
1,20240801_VCP_pratosrefeicoes_piloto_NB_tubercu...,pratosrefeicoes_piloto_NB_tuberculos,8,true,01-08-2024,31-08-2024,0,1,781393,434949#434919#434937#434935,INCLUDED,SUPER_DISCOUNT,0
2,20240801_VCP_pratosrefeicoes_piloto_NB_frutas_W31,pratosrefeicoes_piloto_NB_frutas,8,true,01-08-2024,31-08-2024,0,1,781378,434987#434895#434925#434983#575766#435001#435005,INCLUDED,SUPER_DISCOUNT,0
3,20240801_VCP_pratosrefeicoes_piloto_NB_suinos_W31,pratosrefeicoes_piloto_NB_suinos,8,true,01-08-2024,31-08-2024,0,1,781391,483989#483953#622149#621479,INCLUDED,SUPER_DISCOUNT,0
4,20240801_VCP_pratosrefeicoes_piloto_NB_tempero...,pratosrefeicoes_piloto_NB_temperos,8,true,01-08-2024,31-08-2024,0,1,781392,430830#651889#640549#652929#531960,INCLUDED,SUPER_DISCOUNT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,20240801_VCP_padariacafebolos_control_NB_utens...,padariacafebolos_control_NB_utensiliosdelimpeza,1,true,01-01-2028,02-01-2028,1,1,781412,441499#430497,INCLUDED,SUPER_DISCOUNT,0
268,20240801_VCP_padariacafebolos_control_NB_sucos...,padariacafebolos_control_NB_sucosechas,1,true,01-01-2028,02-01-2028,1,1,781409,455349#430371#430375,INCLUDED,SUPER_DISCOUNT,0
269,20240801_VCP_padariacafebolos_control_NB_carne...,padariacafebolos_control_NB_carnevermelha,1,true,01-01-2028,02-01-2028,1,1,781660,596023#596005,INCLUDED,SUPER_DISCOUNT,0
270,20240801_VCP_padariacafebolos_control_NB_mante...,padariacafebolos_control_NB_manteigasemargarinas,1,true,01-01-2028,02-01-2028,1,1,781675,476629#476637#476633#476625,INCLUDED,SUPER_DISCOUNT,0


In [248]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'Growth/SD/export_data/MS/csv_{city_code}_All_MS.csv', index = False)

# for i,offer in enumerate(len_offer):
#     offer.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS_{i}.csv', index = False)

# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#         json.dump(global_segment_list, outfile)    
# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
#         json.dump(global_segment_list, outfile)
print(f'Done {city_code}')

Done VCP
